In [4]:
import argparse
import torch
import configs.mamba as cm
import configs.xlstm as cx
import configs.transformer as ct
import configs.common as cc
import models
import math

# def train(type, name):
#     if type == "mamba":
#         model = models.mamba.Mamba()
#         pass
#     elif type == "xlstm":
#         model = models.xlstm.xLSTM()
#         pass
#     elif type == "transformer":
#         model = models.transformer.Transformer()
#         pass
#     # loaded_model = torch.load(f'models/{model}/{name}.pth')
#     # model.load_state_dict(state_dict)




# if __name__ == "__main__":
#     parser = argparse.ArgumentParser(description="Training Script")

#     # Add command-line arguments
#     parser.add_argument("--model",
#                     type=str,
#                     default="mamba",
#                     choices=["mamba", "xlstm", "transformer"],  # List of allowed choices
#                     help="Model name (choices: mamba, xlstm, transformer)")
#     parser.add_argument("--name", type=str, help="Name of the model to train, e.g. mamba1000.pth")
#     # parser.add_argument("--batch_size", type=int, default=32, help="Batch size")
#     # parser.add_argument("--learning_rate", type=float, default=0.001, help="Learning rate")

#     # Parse arguments
#     args = parser.parse_args()

#     # Call training function
#     train(args.model, args.name)




In [5]:
config_values = cm.config_dict['model_values']
config_values['vocab_size'] = cc.vocab_size
config_values['d_inner'] = int(config_values['expand'] * config_values['d_model'])
config_values['dt_rank'] = math.ceil(config_values['d_model'] / 16)
    
if config_values['vocab_size'] % config_values['pad_vocab_size_multiple'] != 0:
    config_values['vocab_size'] += (config_values['pad_vocab_size_multiple'] - config_values['vocab_size'] % config_values['pad_vocab_size_multiple'])

model = models.mamba.Mamba(config_values)


In [53]:
parser = argparse.ArgumentParser()
for key, value in cm.config_dict['model_values'].items():
    print(value)
    parser.add_argument(f'--{key}', value)

12


TypeError: 'int' object is not subscriptable

In [54]:
cm.config_dict['model_values']

{'d_model': 12,
 'n_layer': 4,
 'd_state': 8,
 'expand': 4,
 'dt_rank': 64,
 'd_conv': 6,
 'pad_vocab_size_multiple': 1,
 'conv_bias': True,
 'bias': False}